# Simulation of a single freely-jointed chain

In [1]:
import unittest
import gen_chain
import math
import feasst as fst
print(fst.version())

def end_to_end_distance(mc):
    config = mc.system().configuration()
    pos = config.particle(0).site(0).position()
    return pos.distance(config.particle(0).site(config.particle(0).num_sites() - 1).position())

def radius_of_gyration(mc):
    config = mc.system().configuration()
    r_cm = fst.Position(config.dimension())
    n = config.particle(0).num_sites()
    for i in range(n):
        r_cm.add(config.particle(0).site(i).position())
    r_cm.divide(n)
    rg = 0
    for i in range(n):
        rg += config.particle(0).site(i).position().squared_distance(r_cm)
    return rg/n
        
class TestFreelyJointedIdealChain(unittest.TestCase):
    def test(self):
        n=10000
        gen_chain.linear_single_site(n, "data.chain")
        mc = fst.MonteCarlo()
        mc.set(fst.MakeRandomMT19937(fst.args({"seed": "time"})))
        mc.add(fst.Configuration(fst.MakeDomain(fst.args({"cubic_box_length": str(3*n)})),
                                 fst.args({"particle_type": "data.chain"})))
        mc.add(fst.MakePotential(fst.DontVisitModel()))
        mc.set(fst.MakeThermoParams(fst.args({"beta": "1", "chemical_potential0": "1"})))
        mc.set(fst.MakeMetropolis())
        fst.SeekNumParticles(1).with_trial_add().run(mc)
        mc.add(fst.MakeTrialGrowLinear(fst.MakeTrialComputeMove(), fst.args({"particle_type": "0"})))
        steps_per = str(int(1))
        mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "chain"})))
        mc.add(fst.MakeCheckRigidBonds(fst.args({"steps_per": steps_per})))
        end_to_end = fst.Accumulator()
        rg2 = fst.Accumulator()
        for i in range(int(1e2)):
            mc.attempt()
            end_to_end.accumulate(end_to_end_distance(mc))
            rg2.accumulate(radius_of_gyration(mc))
        print(end_to_end.str())
        print(rg2.str())
        self.assertAlmostEqual(end_to_end.average(), math.sqrt(n), delta=20)
        self.assertAlmostEqual(rg2.average(), n/6, delta=200)

v0.16.0-4-g799a04797d-hwh/chaintutorial


In [2]:
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestFreelyJointedIdealChain) ... 

average,stdev,block_stdev,n,moment0,moment1,
92.762178453869083,41.820156853843137,0,100,9276.2178453869082,1033625.8015594697,
average,stdev,block_stdev,n,moment0,moment1,
1838.1569029766988,1036.4719372753318,0,100,183815.69029766987,444235213.59525753,


ok

----------------------------------------------------------------------
Ran 1 test in 17.671s

OK
